# Основы Pandas

## Загрузка данных

In [50]:
import numpy as np
import pandas as pd

In [114]:
df = pd.read_csv('ds_salaries_mod.csv')
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847.0,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000.0,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500.0,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000.0,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000.0,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000.0,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000.0,US,100,US,L
3752,2020,EN,FT,Data Scientist,105000,USD,105000.0,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000.0,US,100,US,L


# Домашнее задание

## Easy

Изучите распределение размеров компаний (`company_size`) с помощью метода `.value_counts()`. Опишите своими словами, что вы увидели.

In [104]:
# Ваш код ниже:
df.value_counts("company_size")

company_size
M    3153
L     454
S     148
Name: count, dtype: int64

Вывод: Самые распространенные это M, далее идут L и самое маленькое количество компаний S.

## Normal

Сделайте выборку за 2023 год. Постройте по ней сводную таблицу, сгруппированную по размеру компании-работодателя (`company_size`), со средней и медианной зарплатой в долларах. Не забывайте также указать количество.

In [105]:
# Ваш код ниже:
df_2023 = df[df["work_year"] == 2023]
df_2023.groupby("company_size")['salary_in_usd'].agg(['count', 'mean', 'median'])

,count,mean,median
company_size,,,
L,71,138622.845070,126100.0
M,1275,151126.316078,145000.0
S,17,73632.647059,53654.0


Напишите краткий вывод из полученной таблицы:

Вывод: Количество повторяет вывод в задаче easy.
Из-за малого числа компаний S врядли можно считать данные о зп в маленьких компаниях надежными.
Наиболее большие зарплаты в средних компаниях.
Зарплаты в маленьких компаниях сильно меньше. 
Для компаний L, M медиана и среднее приблизительно одинаковы.
Для S же они ощутимо отличаются.

Аналогично исследуйте 2022 год. Есть ли отличия?

In [106]:
# Ваш код ниже:
df_2022 = df[df["work_year"] == 2022]
df_2022.groupby("company_size")['salary_in_usd'].agg(['count', 'mean', 'median'])

,count,mean,median
company_size,,,
L,143,120581.482517,120000.0
M,1096,137245.859489,135000.0
S,40,75092.325000,58000.0


Вывод: Данные о компаниях L и M в целом похожи, наблюдается рост зп от 22 года к 23.
Для компаний S данный тоже похожи между годами, что дает возможность с большей уверенностью сказать, зарплаты в маленьких компаниях сильно меньше чем в L и M.
Так же наблюдается падение зп 22 года к 23 в компаниях S.

## Hard

Для начала попробуйте восстановить пропущенные значения `salary_in_usd`.

<details>
<summary>Подсказка (нажмите, чтобы посмотреть):</summary>
Один из способов:
    <ul><li>создайте дополнительный столбец с "курсом доллара" для каждого наблюдения;</li>
    <li>проверьте, менялся ли курс каждой валюты в течение года;</li>
    <li>по возможности примените этот курс для расчета пропущенного значения;</li>
    <li>если остались пропуски, которые нельзя заполнить, удалите их.</li></ul>    
</details>

Оставляйте комментарии в коде своего решения.

In [107]:
# Ваш код:
df["ex_rate"] = df["salary_in_usd"] / df["salary"]

# словари со средними курсами валют(за год и за все время)
ex_rates_by_year = df.groupby(["work_year", "salary_currency"])["ex_rate"].mean().to_dict()
ex_rates = df.groupby(["salary_currency"])["ex_rate"].mean().to_dict()

import math

def set_empty_salary_in_usd(row):
    if math.isnan(row.salary_in_usd):
        ex_rate = ex_rates_by_year[(row.work_year, row.salary_currency)]
        if math.isnan(ex_rate):
            ex_rate = ex_rates[row.salary_currency]
        return ex_rate * row.salary
    return row.salary_in_usd

df["salary_in_usd"] = df.apply(set_empty_salary_in_usd, axis=1)
df = df.dropna(subset=["salary_in_usd"])
df


,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,ex_rate
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847.0,ES,100,ES,L,1.073087
1,2023,MI,CT,ML Engineer,30000,USD,30000.0,US,100,US,S,1.000000
2,2023,MI,CT,ML Engineer,25500,USD,25500.0,US,100,US,S,1.000000
3,2023,SE,FT,Data Scientist,175000,USD,175000.0,CA,100,CA,M,1.000000
4,2023,SE,FT,Data Scientist,120000,USD,120000.0,CA,100,CA,M,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000.0,US,100,US,L,1.000000
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000.0,US,100,US,L,1.000000
3752,2020,EN,FT,Data Scientist,105000,USD,105000.0,US,100,US,S,1.000000
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000.0,US,100,US,L,1.000000


Дополнительно: попробуйте свести данные в столбце `job_title` к 4 основным профессиям (Data Scientist, Data Analyst, Data Engineer, ML Engineer), где это возможно.

In [136]:
# Ваш код:
def unify_job_title(name):
    if (
        "Data" in name
        and (
            "Engineer" in name
            or "Architect" in name
            or "Manager" in name
            or "Head" in name
            or "Modeler" in name
            or "Strategist" in name
            or "Specialist" in name
            or "Lead" in name
            or "Management" in name
        )
    ) or "ETL" in name:
        return "Data Engineer"
    elif (
        "Scientist" in name
        or "Science" in name
        or "Research Engineer" in name
    ):
        return "Data Scientist"
    elif (
        "Analyst" in name
        or "Analytics" in name
        or "BI" in name
        or "Business Intelligence" in name
    ):
        return "Data Analyst"
    elif (
        "Machine Learning" in name
        or "ML Engineer" in name
        or "NLP" in name
        or "Deep Learning" in name
        or "Computer Vision" in name
        or "AI" in name
        or "MLOps" in name
        or "Autonomous Vehicle Technician" in name
    ):
        return "ML Engineer"
    else:
        return name

df["job_title"] = df["job_title"].apply(unify_job_title)
df["job_title"].value_counts()

job_title
Data Engineer     1276
Data Scientist    1209
Data Analyst       839
ML Engineer        431
Name: count, dtype: int64

Сделайте выборку за 2023 год. Постройте по ней сводную таблицу, сгруппированную по профессии (`job_title`), со средней и медианной зарплатой в долларах. Не забывайте также указать количество. (Если у вас не получилось привести в порядок столбец `job_title`, выведите только 4 самых распространенных профессии.)

In [137]:
# Ваш код:
df[df["work_year"] == 2023].groupby("job_title").agg("mean", "median", "count")


,work_year,salary,salary_in_usd,remote_ratio
job_title,,,,
Data Analyst,2023.0,133412.495050,121472.870253,38.490099
Data Engineer,2023.0,160233.755738,151353.939203,35.573770
Data Scientist,2023.0,179115.153707,162202.373171,31.283906
ML Engineer,2023.0,163252.403670,166848.756250,28.669725


Теперь давайте получше категоризируем зарплаты. Изучите самостоятельно документацию к функциям pandas [cut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html) и [qcut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html). Используйте их, чтобы разбить зарплаты на три или больше категорий (например: `Low`, `Medium`, `High`) и запишите их в столбец `salary_range`.

Сделайте сводную таблицу по профессиям за 2023 год, где будут выведены:
- число работников в профессии;
- самая часто встречающаяся категория зарплаты.

<details>
<summary>Подсказка (нажмите, чтобы посмотреть):</summary>
<code>lambda x : pd.Series.mode(x)</code>
</details>

In [155]:
# Ваш код:
df["salary_range"] = pd.cut(df["salary_in_usd"], 3, labels=["L", "M", "H"])
df[df["work_year"] == 2023].groupby("job_title")["salary_range"].agg(["count", lambda x : pd.Series.mode(x)])

,count,<lambda_0>
job_title,,
Data Analyst,316,L
Data Engineer,477,L
Data Scientist,410,M
ML Engineer,160,L


Проверьте себя: совпадают ли категории с реальными цифрами?

In [159]:
df[df["work_year"] == 2023].groupby("job_title")["salary_range"].value_counts()
# Выводы верны.

job_title       salary_range
Data Analyst    L               239
                M                76
                H                 1
Data Engineer   L               271
                M               200
                H                 6
Data Scientist  M               205
                L               197
                H                 8
ML Engineer     L                88
                M                66
                H                 6
Name: count, dtype: int64